# LSTM Using Keras

In [2]:
%pip install tensorflow keras

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 112.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 113.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 133.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 127.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 KB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.8/167.8 KB 33.4 MB/s eta 0:00:00
 

In [3]:
# library import
import os
import lightgbm as lgb
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
import seaborn as sns

pd.set_option('display.max_columns',150)
pd.set_option('display.max_rows',50)

os.chdir('ml4t')

In [4]:
df = pd.read_parquet('data/us_equity/taq_min/AAPL.parquet')
df.head()

,TradeDate,BarDateTime,Ticker,SecId,OpenBarTimeOffset,OpenBidPrice,OpenBidSize,OpenAskPrice,OpenAskSize,FirstTradeTimeOffset,FirstTradePrice,FirstTradeSize,HighBidTimeOffset,HighBidPrice,HighBidSize,HighAskTimeOffset,HighAskPrice,HighAskSize,HighTradeTimeOffset,HighTradePrice,HighTradeSize,LowBidTimeOffset,LowBidPrice,LowBidSize,LowAskTimeOffset,LowAskPrice,LowAskSize,LowTradeTimeOffset,LowTradePrice,LowTradeSize,CloseBarTimeOffset,CloseBidPrice,CloseBidSize,CloseAskPrice,CloseAskSize,LastTradeTimeOffset,LastTradePrice,LastTradeSize,MinSpread,MaxSpread,CancelSize,VolumeWeightPrice,NBBOQuoteCount,TradeAtBid,TradeAtBidMid,TradeAtMid,TradeAtMidAsk,TradeAtAsk,TradeAtCrossOrLocked,Volume,TotalTrades,FinraVolume,FinraVolumeWeightPrice,UptickVolume,DowntickVolume,RepeatUptickVolume,RepeatDowntickVolume,UnknownTickVolume,TradeToMidVolWeight,TradeToMidVolWeightRelative,TimeWeightBid,TimeWeightAsk
0,2012-01-03,2012-01-03 04:00:00-05:00,AAPL,0,0,0.00,0,0.00,0,0.900000000,404.8000,100,0.886000000,410.18,200,0.705000000,412.00,600,25.310000000,412.0000,159,0.705000000,410.10,400,0.705000000,412.00,600,0.900000000,404.8000,100,59.999000000,410.13,200,412.00,300,48.680000000,412.0000,100,1.82,1.90,0,408.26676,304,2500,0,0,0,259,0,2759,10,0,0.00000,659,0,1100,0,1000,-282.12849,-1.55123,410.13000,412.00000
1,2012-01-03,2012-01-03 04:01:00-05:00,AAPL,0,0,410.13,200,412.00,300,0E-9,0.0000,0,40.727000000,410.28,200,0E-9,412.00,300,0E-9,0.0000,0,2.073000000,410.10,800,0E-9,412.00,300,0E-9,0.0000,0,59.999000000,410.21,100,412.00,300,0E-9,0.0000,0,1.72,1.90,0,0.00000,196,0,0,0,0,0,0,0,0,0,0.00000,0,0,0,0,0,0.00000,0.00000,410.21000,412.00000
2,2012-01-03,2012-01-03 04:02:00-05:00,AAPL,0,0,410.21,100,412.00,300,0E-9,0.0000,0,0E-9,410.21,100,0E-9,412.00,300,0E-9,0.0000,0,0E-9,410.21,100,0E-9,412.00,300,0E-9,0.0000,0,59.999000000,410.21,100,412.00,300,0E-9,0.0000,0,1.79,1.79,0,0.00000,0,0,0,0,0,0,0,0,0,0,0.00000,0,0,0,0,0,0.00000,0.00000,410.21000,412.00000
3,2012-01-03,2012-01-03 04:03:00-05:00,AAPL,0,0,410.21,100,412.00,300,0E-9,0.0000,0,0E-9,410.21,100,0E-9,412.00,300,0E-9,0.0000,0,0E-9,410.21,100,0E-9,412.00,300,0E-9,0.0000,0,59.999000000,410.21,100,412.00,300,0E-9,0.0000,0,1.79,1.79,0,0.00000,0,0,0,0,0,0,0,0,0,0,0.00000,0,0,0,0,0,0.00000,0.00000,410.21000,412.00000
4,2012-01-03,2012-01-03 04:04:00-05:00,AAPL,0,0,410.21,100,412.00,300,0E-9,0.0000,0,21.061000000,411.00,100,0E-9,412.00,300,0E-9,0.0000,0,0E-9,410.21,100,54.597000000,411.90,200,0E-9,0.0000,0,59.999000000,411.00,100,411.90,200,0E-9,0.0000,0,0.90,1.79,0,0.00000,4,0,0,0,0,0,0,0,0,0,0.00000,0,0,0,0,0,0.00000,0.00000,411.00000,411.90000


In [10]:
dataset = df['LastTradePrice'].values

In [12]:
dataset = dataset.reshape(-1, 1)

In [14]:
dataset_train = np.array(dataset[:int(dataset.shape[0]*0.8)])
dataset_test = np.array(dataset[int(dataset.shape[0]*0.8):])
scaler = MinMaxScaler(feature_range=(0,1))
dataset_train = scaler.fit_transform(dataset_train)
dataset_test = scaler.transform(dataset_test)

In [15]:
def create_dataset(df):
    x = []
    y = []
    for i in range(50, df.shape[0]):
        x.append(df[i-50:i, 0])
        y.append(df[i, 0])
    x = np.array(x)
    y = np.array(y)
    return x,y

In [16]:
x_train, y_train = create_dataset(dataset_train)
x_test, y_test = create_dataset(dataset_test)

In [17]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [18]:

model = Sequential()
model.add(LSTM(units=96, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=96,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=96,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=96))
model.add(Dropout(0.2))
model.add(Dense(units=1))

2022-07-30 10:17:25.985364: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-30 10:17:25.985422: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-10-80): /proc/driver/nvidia/version does not exist
2022-07-30 10:17:25.987824: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [20]:
model.compile(loss='mean_squared_error', optimizer='adam')


In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=32)


In [ ]:
import matplotlib.pyplot as plt
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
y_test_scaled = scaler.inverse_transform(y_test.reshape(-1, 1))

fig, ax = plt.subplots(figsize=(16,8))
ax.set_facecolor('black')
ax.plot(y_test_scaled, color='red', label='Original price')
plt.plot(predictions, color='cyan', label='Predicted price')
plt.title('Stock price vs Predict price')
plt.legend()